In [1]:
import json 
import pandas as pd
import numpy as np
import datetime
import requests

In [2]:
# Opening JSON file 
f = open('neighbor-districts-modified.json') 
  
# returns JSON object as a dictionary 
data = json.load(f) 

# sorting districts alphabetically
districts_from_json = []
for i in data:
    districts_from_json.append(i)
districts_from_json = np.array(districts_from_json)
districts_from_json.sort()

# ids for districts
ids = np.arange(101,101+len(districts_from_json))

# Dictionary for storing ids of districts
Dict = {} 
for i in range(len(ids)):
    Dict[districts_from_json[i]]=ids[i]

In [3]:
district_names_from_json = []
district_ids_from_json = []
for i in range(len(districts_from_json)):
    district_names_from_json.append(districts_from_json[i].split('/')[0])
    district_ids_from_json.append(districts_from_json[i].split('/')[1])

In [4]:
districts_to_complete_ids = {}
for i in range(len(district_names_from_json)):
    districts_to_complete_ids[district_names_from_json[i]] = district_names_from_json[i] + '/' + district_ids_from_json[i]

In [5]:
time_id_week = {}
time_id_month = {}
time_id_overall = {}
date = datetime.date(2020,3,15)
day = 1
while True:
    time_id_week[str(date)] = int(np.ceil(day/7))
    time_id_month[str(date)] = int(str(date).split('-')[1])-2
    time_id_overall[str(date)] = 1
    if date == datetime.date(2020,9,5):
        break
    day+=1
    date += datetime.timedelta(days=1)

date = datetime.date(2020,3,15)

In [6]:
r = requests.get('https://api.covid19india.org/v4/data-all.json')
dat = json.loads(r.text)

date = datetime.date(2020,3,15)

Date = []
State = []
District = []
Dist_state = []
numcases = []
weekid = []
monthid = []
overallid = []
pop = []

while True:
    for state in dat[str(date)]:
        flag_districts=0
        for key in dat[str(date)][state]:
            if key == 'districts':
                flag_districts=1
                for district in dat[str(date)][state]['districts']:
                    flag_delta=0
                    for key_delta in dat[str(date)][state]['districts'][district]:
                        if key_delta == 'delta':
                            flag_delta=1
                            flag_confimed=0
                            for key_confirmed in dat[str(date)][state]['districts'][district]['delta']:
                                if key_confirmed == 'confirmed':
                                    flag_confirmed=1
                                    Date.append(str(date))
                                    State.append(state)
                                    District.append(district)
                                    Dist_state.append(district+':'+state)
                                    numcases.append(dat[str(date)][state]['districts'][district]['delta']['confirmed'])
                                    weekid.append(time_id_week[str(date)])
                                    monthid.append(time_id_month[str(date)])
                                    overallid.append(time_id_overall[str(date)])
                                    if 'meta' in dat[str(date)][state]['districts'][district].keys():
                                        if 'population' in dat[str(date)][state]['districts'][district]['meta'].keys():
                                            pop.append(dat[str(date)][state]['districts'][district]['meta']['population'])
                                        else:
                                            pop.append(0)
                                    else:
                                        pop.append(0)
        if flag_districts == 0:
            flag_delta=0
            if state== 'TT':
                continue
            for key in dat[str(date)][state]:
                if key == 'delta':
                    falg_delta=1
                    flag_confirmed = 0
                    for key_confirmed in dat[str(date)][state]['delta']:
                        if key_confirmed == 'confirmed':
                            flag_confirmed = 1
                            Date.append(str(date))
                            State.append(state)
                            District.append('All Districts')
                            numcases.append(dat[str(date)][state]['delta']['confirmed'])
                            weekid.append(time_id_week[str(date)])
                            monthid.append(time_id_month[str(date)])
                            overallid.append(time_id_overall[str(date)])
                            Dist_state.append('All Districts:'+state)
                            pop.append(0)
        
    if date == datetime.date(2020,9,5):
        break
    date += datetime.timedelta(days=1)

District_wise_data = {'Date': Date,'State': State, 'District': District, 'District_state': Dist_state, 'cases': numcases, 'weekid': weekid, 'monthid': monthid, 'overallid': overallid, 'Population': pop}
District_wise_data = pd.DataFrame(District_wise_data)
District_wise_data.sort_values("Population", axis = 0, ascending = False, kind='mergesort', inplace = True, na_position ='last')
District_wise_data.sort_values("State", axis = 0, ascending = True, kind='mergesort', inplace = True, na_position ='last')
District_wise_data = District_wise_data.reset_index(drop=True)
# District_wise_data.to_csv('District_wise_data.csv')


In [7]:
populous_district = {}
i=0
while i<len(District_wise_data):
    curr = District_wise_data['State'][i]
    populous_district[District_wise_data['State'][i]] = District_wise_data['District'][i]
    i+=1
    while i<len(District_wise_data) and District_wise_data['State'][i] ==curr:
        i+=1

In [8]:
r = requests.get('https://api.covid19india.org/v4/data-all.json')
dat = json.loads(r.text)

date = datetime.date(2020,3,15)

Date = []
State = []
District = []
Dist_state = []
numcases = []
weekid = []
monthid = []
overallid = []
pop = []

while True:
    for state in dat[str(date)]:
        flag_districts=0
        for key in dat[str(date)][state]:
            if key == 'districts':
                flag_districts=1
                for district in dat[str(date)][state]['districts']:
                    flag_delta=0
                    for key_delta in dat[str(date)][state]['districts'][district]:
                        if key_delta == 'delta':
                            flag_delta=1
                            flag_confimed=0
                            for key_confirmed in dat[str(date)][state]['districts'][district]['delta']:
                                if key_confirmed == 'confirmed':
                                    flag_confirmed=1 
                                    Date.append(str(date))
                                    State.append(state)
                                    District.append(district)
                                    Dist_state.append(district+':'+state)
                                    numcases.append(dat[str(date)][state]['districts'][district]['delta']['confirmed'])
                                    weekid.append(time_id_week[str(date)])
                                    monthid.append(time_id_month[str(date)])
                                    overallid.append(time_id_overall[str(date)])
#                                     Dist_id.append(Dict[districts_to_complete_ids[district.lower()]])
                                    if 'meta' in dat[str(date)][state]['districts'][district].keys():
                                        if 'population' in dat[str(date)][state]['districts'][district]['meta'].keys():
                                            pop.append(dat[str(date)][state]['districts'][district]['meta']['population'])
                                        else:
                                            pop.append(0)
                                    else:
                                        pop.append(0)
        if flag_districts == 0:
            flag_delta=0
            if state== 'TT':
                continue
            for key in dat[str(date)][state]:
                if key == 'delta':
                    falg_delta=1
                    flag_confirmed = 0
                    for key_confirmed in dat[str(date)][state]['delta']:
                        if key_confirmed == 'confirmed':
                            flag_confirmed = 1
                            Date.append(str(date))
                            State.append(state)
                            District.append(populous_district[state])
                            numcases.append(dat[str(date)][state]['delta']['confirmed'])
                            weekid.append(time_id_week[str(date)])
                            monthid.append(time_id_month[str(date)])
                            overallid.append(time_id_overall[str(date)])
                            Dist_state.append(populous_district[state]+':'+state)
#                             Dist_id.append(Dict[districts_to_complete_ids[populous_district[state].lower()]])
                            pop.append(0)
        
    if date == datetime.date(2020,9,5):
        break
    date += datetime.timedelta(days=1)

District_wise_data = {'Date': Date,'State': State, 'District': District, 'District_state': Dist_state, 'cases': numcases, 'weekid': weekid, 'monthid': monthid, 'overallid': overallid, 'Population': pop, 'ID': np.zeros(len(numcases))}
District_wise_data = pd.DataFrame(District_wise_data)
# District_wise_data.to_csv('District_wise_data.csv')


In [9]:
for i in range(len(District_wise_data)):
    if District_wise_data['District'][i].lower()=='hamirpur' and District_wise_data['State'][i] == 'HP':
        District_wise_data['ID'][i] = Dict['hamirpur/316']
    elif District_wise_data['District'][i].lower()=='bilaspur' and District_wise_data['State'][i] == 'CT':
        District_wise_data['ID'][i] = Dict['bilaspur/190']
    elif District_wise_data['District'][i].lower()=='pratapgarh' and District_wise_data['State'][i] == 'UP':
        District_wise_data['ID'][i] = Dict['pratapgarh/546']
    elif District_wise_data['District'][i].lower()=='balrampur' and District_wise_data['State'][i] == 'CT':
        District_wise_data['ID'][i] = Dict['balrampur/148']
    elif District_wise_data['District'][i].lower()=='aurangabad' and District_wise_data['State'][i] == 'BR':
        District_wise_data['ID'][i] = Dict['aurangabad/133']
    elif District_wise_data['District'][i].lower() in districts_to_complete_ids.keys():
        District_wise_data['ID'][i] = Dict[districts_to_complete_ids[District_wise_data['District'][i].lower()]]
    else:
        District_wise_data['ID'][i] = 0

indexNames = District_wise_data[District_wise_data['ID'] == 0].index
District_wise_data.drop(indexNames , inplace=True)
District_wise_data = District_wise_data.reset_index(drop=True)

# District_wise_data.to_csv('District_wise_data.csv')

/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by 

In [10]:
state_data = {'districtid': District_wise_data['ID'], 'State': District_wise_data['State']}
state_data = pd.DataFrame(state_data)

sorted1 = state_data.sort_values("districtid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2 = sorted1.sort_values("State", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2['dropped'] = 0
sorted2 = sorted2.reset_index(drop=True)
for i in range(len(sorted2)-1):
    if sorted2['districtid'][i] == sorted2['districtid'][i+1]:
        sorted2['dropped'][i] = 1
indexNames = sorted2[ sorted2['dropped'] == 1].index
sorted2.drop(indexNames , inplace=True)
sorted2 = sorted2.drop(['dropped'], axis = 1)
sorted2 = sorted2.reset_index(drop=True)

/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [11]:
state_districts = {}
for i in range(len(sorted2)):
    if sorted2['State'][i] in state_districts.keys():
        state_districts[sorted2['State'][i]].append(int(sorted2['districtid'][i]))
    else:
        state_districts[sorted2['State'][i]] = []
        state_districts[sorted2['State'][i]].append(int(sorted2['districtid'][i]))

state_map = {}
for key in state_districts.keys():
    for i in range(len(state_districts[key])):
        state_map[state_districts[key][i]] = state_districts[key].copy()

for key in state_map.keys():
    state_map[key].remove(key)

In [12]:
# week
cases_week = pd.read_csv('cases-week.csv')
cases_week = cases_week.drop(['Unnamed: 0'], axis = 1)

weeks = int(cases_week['weekid'].max())

Data1 = {'districtid': [],'weekid': [],'statemean': [], 'statestdev': [] }
state_week = pd.DataFrame(Data1)

for i in range(1,weeks+1):
    start = len(state_map)*(i-1)
    end = len(state_map)*i
    numcases = {}
    for j in range(start,end):
        numcases[int(cases_week['districtid'][j])] = int(cases_week['cases'][j])
    statemean = []
    statestdev = []
    for j in range(101,101+len(state_map)):
        statecases = []
        for k in range(len(state_map[j])):
            statecases.append(numcases[state_map[j][k]])
        statecases = np.array(statecases)
        if len(statecases)>0:
            statemean.append(np.mean(statecases))
            statestdev.append(np.std(statecases))
        else:
            statemean.append(0)
            statestdev.append(0)

        
    Data1 = {'districtid': np.arange(101,101+len(districts_from_json)),'weekid': i*np.ones(len(districts_from_json)),'statemean': statemean, 'statestdev': statestdev }
    Data1 = pd.DataFrame(Data1)
    state_week = state_week.append(Data1)

state_week = state_week.reset_index(drop=True)
state_week.to_csv('state-week.csv')

In [13]:
# month
cases_month = pd.read_csv('cases-month.csv')
cases_month = cases_month.drop(['Unnamed: 0'], axis = 1)

months = int(cases_month['monthid'].max())

Data1 = {'districtid': [],'monthid': [],'statemean': [], 'statestdev': [] }
state_month = pd.DataFrame(Data1)

for i in range(1,months+1):
    start = len(state_map)*(i-1)
    end = len(state_map)*i
    numcases = {}
    for j in range(start,end):
        numcases[int(cases_month['districtid'][j])] = int(cases_month['cases'][j])
    statemean = []
    statestdev = []
    for j in range(101,101+len(state_map)):
        statecases = []
        for k in range(len(state_map[j])):
            statecases.append(numcases[state_map[j][k]])
        statecases = np.array(statecases)
        if len(statecases)>0:
            statemean.append(np.mean(statecases))
            statestdev.append(np.std(statecases))
        else:
            statemean.append(0)
            statestdev.append(0)
        
    Data1 = {'districtid': np.arange(101,101+len(districts_from_json)),'monthid': i*np.ones(len(districts_from_json)),'statemean': statemean, 'statestdev': statestdev }
    Data1 = pd.DataFrame(Data1)
    state_month = state_month.append(Data1)

state_month = state_month.reset_index(drop=True)
state_month.to_csv('state-month.csv')

In [14]:
# overall
cases_overall = pd.read_csv('cases-overall.csv')
cases_overall = cases_overall.drop(['Unnamed: 0'], axis = 1)

overalls = int(cases_overall['overallid'].max())

Data1 = {'districtid': [],'overallid': [],'statemean': [], 'statestdev': [] }
state_overall = pd.DataFrame(Data1)

for i in range(1,overalls+1):
    start = len(state_map)*(i-1)
    end = len(state_map)*i
    numcases = {}
    for j in range(start,end):
        numcases[int(cases_overall['districtid'][j])] = int(cases_overall['cases'][j])
    statemean = []
    statestdev = []
    for j in range(101,101+len(state_map)):
        statecases = []
        for k in range(len(state_map[j])):
            statecases.append(numcases[state_map[j][k]])
        statecases = np.array(statecases)
        if len(statecases)>0:
            statemean.append(np.mean(statecases))
            statestdev.append(np.std(statecases))
        else:
            statemean.append(0)
            statestdev.append(0)
        
    Data1 = {'districtid': np.arange(101,101+len(districts_from_json)),'overallid': i*np.ones(len(districts_from_json)),'statemean': statemean, 'statestdev': statestdev }
    Data1 = pd.DataFrame(Data1)
    state_overall = state_overall.append(Data1)

state_overall = state_overall.reset_index(drop=True)
state_overall.to_csv('state-overall.csv')